## 1. Data

### Data Preprocessing

In [374]:
import pandas as pd
import networkx as nx
import string
import numpy as np

In [375]:
edges = pd.read_csv('edges.csv')

In [376]:
hero_network = pd.read_csv('hero-network.csv')

In [377]:
nodes = pd.read_csv('nodes.csv')

Removing inconsistencies

In [378]:
#Removing extra spaces at the end of the hero name
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())

In [379]:
#Replace 'SPIDER-MAN/PETER PAR' with 'SPIDER-MAN/PETER PARKER'
hero_network.replace({'SPIDER-MAN/PETER PAR': 'SPIDER-MAN/PETER PARKER'}, inplace = True)

In [380]:
#Removing extra '/'
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip('/'))
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip('/'))
edges['hero'] = edges['hero'].apply(lambda x: x.strip('/'))

In [381]:
#After this we need to remove again the empty spaces at the end
hero_network['hero1'] = hero_network['hero1'].apply(lambda x: x.rstrip())
hero_network['hero2'] = hero_network['hero2'].apply(lambda x: x.rstrip())
edges['hero'] = edges['hero'].apply(lambda x: x.rstrip())

Removing duplicates

Let's remove rows where hero1 is equal to hero2 first.

In [382]:
hero_network.drop((hero_network[hero_network['hero1'] == hero_network['hero2']]).index, inplace = True)

In [383]:
hero_network = hero_network.apply(lambda row: sorted(row), axis=1, result_type='expand').set_axis(hero_network.columns, axis=1)


### Graphs setup

##### First graph

In [384]:
hero_network.groupby(['hero1', 'hero2']).size()


hero1                 hero2               
24-HOUR MAN/EMMANUEL  FROST, CARMILLA         1
                      G'RATH                  1
                      KILLRAVEN/JONATHAN R    1
                      M'SHULLA                1
                      OLD SKULL               1
                                             ..
ZEFRA                 ZON                     1
ZEUS                  ZIRAN                   1
                      ZURAS                   4
ZIRAN                 ZON                     1
                      ZURAS                   1
Length: 167100, dtype: int64

In [385]:
hero_network = hero_network.groupby(['hero1', 'hero2']).size().reset_index().rename(columns={0: 'size'})

In [386]:
hero_network

,hero1,hero2,size
0,24-HOUR MAN/EMMANUEL,"FROST, CARMILLA",1
1,24-HOUR MAN/EMMANUEL,G'RATH,1
2,24-HOUR MAN/EMMANUEL,KILLRAVEN/JONATHAN R,1
3,24-HOUR MAN/EMMANUEL,M'SHULLA,1
4,24-HOUR MAN/EMMANUEL,OLD SKULL,1
...,...,...,...
167095,ZEFRA,ZON,1
167096,ZEUS,ZIRAN,1
167097,ZEUS,ZURAS,4
167098,ZIRAN,ZON,1


In [387]:
G_hero_net = nx.Graph()

We want the graph to be weighted and we also want the weight to be lower for heroes with more collaborations, hence we take as weights the reciprocal number of the number of collaboration (the size)

In [388]:
# creating a list of 3d tuples where the first two elements are the nodes and the last is the weight
edges_l = []
hero_network.apply(lambda row : edges_l.append((row['hero1'], row['hero2'], 1/row['size'])), axis = 1)

# creating the weighted graph
G_hero_net.add_weighted_edges_from(edges_l)


In [389]:
nx.info(G_hero_net)

C:\Users\erica\AppData\Local\Temp\ipykernel_6756\1638737127.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G_hero_net)


'Graph with 6421 nodes and 167100 edges'

In [390]:
nx.get_edge_attributes(G_hero_net, 'weight')

{('24-HOUR MAN/EMMANUEL', 'FROST, CARMILLA'): 1.0,
 ('24-HOUR MAN/EMMANUEL', "G'RATH"): 1.0,
 ('24-HOUR MAN/EMMANUEL', 'KILLRAVEN/JONATHAN R'): 1.0,
 ('24-HOUR MAN/EMMANUEL', "M'SHULLA"): 1.0,
 ('24-HOUR MAN/EMMANUEL', 'OLD SKULL'): 1.0,
 ('FROST, CARMILLA', 'ABRAXAS'): 0.3333333333333333,
 ('FROST, CARMILLA', 'ADAM 3,031'): 0.25,
 ('FROST, CARMILLA', 'ATALON'): 0.25,
 ('FROST, CARMILLA', 'BROTHER AXE'): 1.0,
 ('FROST, CARMILLA', 'DEATHRAVEN/JOSHUA RA'): 0.5,
 ('FROST, CARMILLA', 'DEVOURER'): 1.0,
 ('FROST, CARMILLA', 'EVE 3,031'): 0.25,
 ('FROST, CARMILLA', 'FOROPULIST'): 0.5,
 ('FROST, CARMILLA', "G'RATH"): 1.0,
 ('FROST, CARMILLA', 'GROK'): 0.07692307692307693,
 ('FROST, CARMILLA', 'HAWK'): 0.07692307692307693,
 ('FROST, CARMILLA', 'HIGH OVERLORD'): 0.16666666666666666,
 ('FROST, CARMILLA', 'KILLRAVEN/JONATHAN R'): 0.05555555555555555,
 ('FROST, CARMILLA', "M'SHULLA"): 0.05555555555555555,
 ('FROST, CARMILLA', 'MELONIE'): 0.5,
 ('FROST, CARMILLA', 'MILLER, JENETTE'): 1.0,
 ('FROST, 

##### Second graph

In [391]:
nodes

,node,type
0,2001 10,comic
1,2001 8,comic
2,2001 9,comic
3,24-HOUR MAN/EMMANUEL,hero
4,3-D MAN/CHARLES CHAN,hero
...,...,...
19085,"ZOTA, CARLO",hero
19086,ZOTA,hero
19087,ZURAS,hero
19088,ZURI,hero


In [392]:
edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


In [393]:
G_comic_net = nx.Graph()

In [394]:
G_comic_net.add_nodes_from(list(nodes['node']))

edges_list= []
edges.apply(lambda row: edges_list.append((row['hero'], row['comic'])), axis = 1)

G_comic_net.add_edges_from(edges_list)

In [395]:
nx.info(G_comic_net)

C:\Users\erica\AppData\Local\Temp\ipykernel_6756\3041480722.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G_comic_net)


'Graph with 19519 nodes and 96104 edges'

## 2. Backend Implementation

Top N heroes: The top N heroes who have appeared in the most number of comics. The 'edges.csv' file, which represents the comics in which each hero has appeared, can be used to filter these N heroes.

In [396]:
N = 10
topN = edges.groupby('hero').count().sort_values(by = 'comic', ascending = False).head(N)

In [397]:
topN

,comic
hero,
SPIDER-MAN/PETER PARKER,1577
CAPTAIN AMERICA,1334
IRON MAN/TONY STARK,1150
THING/BENJAMIN J. GR,963
THOR/DR. DONALD BLAK,956
HUMAN TORCH/JOHNNY S,886
MR. FANTASTIC/REED R,854
HULK/DR. ROBERT BRUC,835
WOLVERINE/LOGAN,819


In [398]:
topN = topN.reset_index()
topN

,hero,comic
0,SPIDER-MAN/PETER PARKER,1577
1,CAPTAIN AMERICA,1334
2,IRON MAN/TONY STARK,1150
3,THING/BENJAMIN J. GR,963
4,THOR/DR. DONALD BLAK,956
5,HUMAN TORCH/JOHNNY S,886
6,MR. FANTASTIC/REED R,854
7,HULK/DR. ROBERT BRUC,835
8,WOLVERINE/LOGAN,819
9,INVISIBLE WOMAN/SUE,762


### Functionality 2 - Find top superheroes!


In [399]:
def functionality2(G, node, metric, N):
     
     topN = edges.groupby('hero').count().sort_values(by = 'comic', ascending = False).head(N)
     topN = topN.reset_index()

     if G == G_hero_net:
          subG = G.subgraph(list(topN['hero']))

          if metric == 'Betweeness':
               gvalue = nx.betweenness_centrality(subG, weight = 'weight')

          elif metric == 'PageRank':
               gvalue = nx.pagerank(subG, weight = 'weight')

          elif metric == 'ClosenessCentrality':
               gvalue = nx.closeness_centrality(subG, distance = 'weight')

          elif metric == 'DegreeCentrality':
               gvalue = nx.degree_centrality(subG)

          if node in subG.nodes:
               nvalue = gvalue[node]
          else:
               nvalue = 'ERROR: node not in the subgraph'

          return gvalue, nvalue

     if G == G_comic_net:

          reduced_edges = []
          for n in topN['hero']:
               for elem in G.edges(n):
                    reduced_edges.append(elem)
          subG = nx.Graph()
          subG.add_edges_from(set(reduced_edges))
          
          if metric == 'Betweeness':
               gvalue = nx.betweenness_centrality(subG)

          elif metric == 'PageRank':
               gvalue = nx.pagerank(subG)

          elif metric == 'ClosenessCentrality':
               gvalue = nx.closeness_centrality(subG)

          elif metric == 'DegreeCentrality':
               gvalue = nx.degree_centrality(subG)

          if node in subG.nodes:
               nvalue = gvalue[node]
          else:
               nvalue = 'ERROR: node not in the subgraph'

          return gvalue, nvalue

     

In [400]:
node = 'CAPTAIN AMERICA'
g, n = functionality2(G_hero_net, node, 'PageRank', 100)

g = pd.DataFrame.from_dict(g, orient ='index', columns = ["metric's value"])
print(n)

0.004224894301898034


### Functionality 4 - Disconnecting Graphs

In [ ]:
def functionality4(G, heroA, heroB, N):
    topN = edges.groupby('hero').count().sort_values(by = 'comic', ascending = False).head(N)
    topN = topN.reset_index()
    #per il momento considero di passare solo il primo grafo 
    subG = G.subgraph(list(topN['hero']))
    